In [11]:
import requests
import json
from tqdm import tqdm
import os
import pandas as pd
import ast
import random

from dotenv import load_dotenv
load_dotenv()

True

In [78]:
url = "https://exodus.stockbit.com/screener/templates"
headers = {"Authorization": os.getenv("BEARER_TOKEN")}

In [186]:
df_payload = pd.read_csv("data/Stock Screeners - Payload.csv")
df_payload['payload'] = df_payload['payload'].apply(lambda x: ast.literal_eval(x))

payloads = df_payload['payload']
ratios = df_payload['ratio']

In [187]:
for ratio, payload in zip(ratios, payloads):
    result = []
    print(ratio)
    for i in tqdm(range(36)):
        payload['page'] = i + 1
        r = requests.post(url=url, headers=headers, data=json.dumps(payload))
        content = json.loads(r.content.decode('utf-8'))
        result.extend(content['data']['calcs'])
    
    with open(f'data/ratios/{ratio}.json', 'w') as f:
        json.dump(result, f)

market_cap


100%|██████████| 36/36 [00:14<00:00,  2.50it/s]


enterprise_value


100%|██████████| 36/36 [00:14<00:00,  2.41it/s]


current_shares_outstanding


...

100%|██████████| 36/36 [00:14<00:00,  2.54it/s]


eps_growth


100%|██████████| 36/36 [00:13<00:00,  2.74it/s]


sales_growth


100%|██████████| 36/36 [00:14<00:00,  2.51it/s]


In [189]:
data_ratios = dict()

for f in tqdm(os.listdir('data/ratios')):
    with open(f'data/ratios/{f}', 'r') as file:
        data_ratios[f[:-5]] = json.load(file)

100%|██████████| 12/12 [00:00<00:00, 61.82it/s]


In [196]:
cols = [
    'Ticker Code',
    'Current Ratio (Quarter)',
    'Current Share Outstanding',
    'Debt to Equity Ratio (Quarter)',
    'Enterprise Value',
    'EPS Growth Streak',
    'Market Cap',
    'Net Profit Margin (TTM)(%)',
    'Current Price to Book Value',
    'Current PE Ratio (TTM)',
    'Current Price To Free Cashflow (TTM)',
    'Return on Equity (TTM)',
    'Sales Growth Streak'
]

In [203]:
alternatives = ['ADRO', 'PWON', 'TLKM', 'INDF', 'SMGR']

In [204]:
df_alternatives = pd.DataFrame(columns=cols)
for a in alternatives:
    print(a)
    temp_dict = {
        'Ticker Code': [a]
    }
    for key in tqdm(data_ratios.keys()):
        for i in range(len(data_ratios[key])):
            if data_ratios[key][i]['company']['symbol'] == a:
                val = data_ratios[key][i]['results'][0]['display']
                item_name = data_ratios[key][i]['results'][0]['item']

                temp_dict[item_name] = val
        
    temp = pd.DataFrame(temp_dict)
    df_alternatives = pd.concat([df_alternatives, temp])

ADRO


100%|██████████| 12/12 [00:00<00:00, 3418.34it/s]


PWON


100%|██████████| 12/12 [00:00<00:00, 10852.02it/s]


TLKM


100%|██████████| 12/12 [00:00<00:00, 1807.11it/s]


INDF


100%|██████████| 12/12 [00:00<00:00, 1290.13it/s]


SMGR


100%|██████████| 12/12 [00:00<00:00, 7950.03it/s]


# Sektor

In [264]:
sectors = {
    'Barang Konsumen Primer' : 1, 
    'Kesehatan' : 2, 
    'Keuangan' : 3, 
    'Barang Konsumen Non-Primer' : 4, 
    'Properti & Real Estat' : 5, 
    'Perindustrian' : 6, 
    'Energi': 7, 
    'Barang Baku' : 8, 
    'Infrastruktur' : 9, 
    'Teknologi' : 50, 
    'Transportasi & Logistik' : 51
}

In [314]:
series_sector = []
series_ticker = []
for id_sec, s in tqdm(zip(sectors.values(), sectors.keys())):
    url = "https://exodus.stockbit.com/emitten/sectors/{}/subsectors"
    r = requests.get(url.format(id_sec), headers=headers)
    r = json.loads(r.content.decode('utf-8'))
    id_subsectors = [r['data'][j]['id'] for j in range(len(r['data']))]

    for id_sub in id_subsectors:
        url = f"https://exodus.stockbit.com/emitten/v3/sector/{id_sec}/subsector/{id_sub}/company"
        r2 = requests.get(url, headers=headers)
        r2 = json.loads(r2.content.decode('utf-8'))
        ticker = [r2['data'][k]['symbol'] for k in range(len(r2['data']))]

        series_sector.extend([s]*len(ticker))
        series_ticker.extend(ticker)

11it [00:15,  1.37s/it]


In [328]:
pd.DataFrame({'sector':series_sector, 'ticker':series_ticker}).to_csv('sector.csv', index=False)

# Index

In [326]:
index_map = {
    'LQ45': 550,
    'IDX30': 559,
    'IDXHIDIV20': 1000000010,
    'IDXBUMN20': 1000000011
}

index_member = {}

for index_name, id_index in tqdm(zip(index_map.keys(), index_map.values())):
    url = 'https://exodus.stockbit.com/emitten/v3/sector/88/subsector/{}/company'
    r = requests.get(url.format(id_index), headers=headers)
    r = json.loads(r.content.decode('utf-8'))
    members = [r['data'][i]['symbol'] for i in range(len(r['data']))]
    index_member[index_name] = members

4it [00:01,  2.67it/s]


In [329]:
with open('data/index.json', 'w') as f:
    json.dump(index_member, f)

In [206]:
def get_alternatives_data(alternatives):
    # Data Ratios (Kuanti)
    data_ratios = dict()

    for f in tqdm(os.listdir('data/ratios')):
        with open(f'data/ratios/{f}', 'r') as file:
            data_ratios[f[:-5]] = json.load(file)

    # Kuali
    with open('data/index.json', 'r') as file:
        index = json.load(file)

    sector = pd.read_csv('data/sector.csv')

    cols = [
        'Ticker Code',
        'Current Ratio (Quarter)',
        'Current Share Outstanding',
        'Debt to Equity Ratio (Quarter)',
        'Enterprise Value',
        'EPS Growth Streak',
        'Market Cap',
        'Net Profit Margin (TTM)(%)',
        'Current Price to Book Value',
        'Current PE Ratio (TTM)',
        'Current Price To Free Cashflow (TTM)',
        'Return on Equity (TTM)',
        'Sales Growth Streak'
    ]

    df_alternatives = pd.DataFrame(columns=cols)
    for a in alternatives:
        print(a)
        temp_dict = {
            'Ticker Code': [a]
        }
        for key in tqdm(data_ratios.keys()):
            for i in range(len(data_ratios[key])):
                if data_ratios[key][i]['company']['symbol'] == a:
                    val = data_ratios[key][i]['results'][0]['display']
                    item_name = data_ratios[key][i]['results'][0]['item']

                    temp_dict[item_name] = val
            
        temp = pd.DataFrame(temp_dict)
        df_alternatives = pd.concat([df_alternatives, temp])

    index_to_append = list()
    sector_to_append = list()
    for i in range(len(alternatives)):
        member_to = []
        for k in index:
            if alternatives[i] in index[k]:
                member_to.append(k)
        if member_to:
            out = random.choice(member_to)
            index_to_append.append(out)
        else:
            index_to_append.append('Others')

        sector_to_append.append(sector[sector['ticker'] == alternatives[i]].iloc[0]['sector'])

    df_alternatives['sector'] = sector_to_append
    df_alternatives['index'] = index_to_append

    return df_alternatives

In [1]:
alternatives = [
  "ADRO",
  "PWON",
  "TLKM",
  "INDF",
  "SMGR"
]

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:00<00:00, 127.00it/s]


ADRO


100%|██████████| 12/12 [00:00<00:00, 2389.35it/s]


PWON


...

100%|██████████| 12/12 [00:00<00:00, 1890.46it/s]


INDF


100%|██████████| 12/12 [00:00<00:00, 1733.18it/s]


SMGR


100%|██████████| 12/12 [00:00<00:00, 1802.32it/s]


In [36]:
df_alternatives

,Ticker Code,Current Ratio (Quarter),Current Share Outstanding,Debt to Equity Ratio (Quarter),Enterprise Value,EPS Growth Streak,Market Cap,Net Profit Margin (TTM)(%),Current Price to Book Value,Current PE Ratio (TTM),Current Price To Free Cashflow (TTM),Return on Equity (TTM),Sales Growth Streak,sector,index
0,ADRO,2.01,31.99 B,0.21,"67,234.67 B",5.00,"86,362.10 B",25.18%,0.83,3.41,9.90,24.24%,5.00,Energi,IDXHIDIV20
0,PWON,5.16,48.16 B,0.32,"22,651.36 B",4.00,"20,419.67 B",33.95%,1.07,9.70,13.93,11.02%,5.00,Properti & Real Estat,Others
0,TLKM,0.78,99.06 B,0.35,"383,255.89 B",9.00,"343,745.89 B",16.46%,2.53,14.00,14.23,18.09%,5.00,Infrastruktur,IDXHIDIV20
0,INDF,1.92,8.78 B,1.10,"133,751.71 B",-,"55,975.22 B",7.29%,0.95,6.87,3.79,13.76%,17.00,Barang Konsumen Primer,IDXHIDIV20
0,SMGR,1.23,6.75 B,0.32,"51,456.67 B",-,"39,839.91 B",5.62%,0.92,18.36,9.99,5.02%,-,Barang Baku,IDXHIDIV20
